<a href="https://colab.research.google.com/github/jj2773/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-23 00:19:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2021-12-23 00:19:08 (4.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [10]:
# Configure settings for RDS
# Note for GITHUB publishing the jdbc_url and config data was changed to XXXXXX
# mode = "read"
jdbc_url="jdbc:postgresql://XXXXXXXXXXamazonaws.com:5432/XXXXXXX_db"
config = {"user":"XXXXXX", 
          "password": "XXXXXXXXXX", 
          "driver":"org.postgresql.Driver"}

In [11]:
vine_df=spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)

In [12]:
# show the dataframe pulled from our AWS Postgres DB [5 pts]
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [14]:
# Review the schema
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [15]:
# Filter the data for more than 20 total votes [5 pts]
vine_gt20_df=vine_df.filter(vine_df["total_votes"]>=20)

In [77]:
vine_gt20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [78]:
# Filter the data again for helpful_votes is greater than or equal to 50% [5pts]
vine_hv_per_totv_df=vine_gt20_df.filter(vine_gt20_df.helpful_votes/vine_gt20_df.total_votes>=.5)

In [79]:
vine_hv_per_totv_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

In [80]:
# calculate the total number of reviews left after required criteria above
total_reviews=vine_hv_per_totv_df.count()

In [81]:
total_reviews

8409

In [82]:
# total number of 5 star reviews
vine_hv_per_totv_df.filter(vine_hv_per_totv_df.star_rating==5).count()

4347

In [83]:
# data frame for vine members Yes [5 pts]
vine_y_df=vine_hv_per_totv_df.filter(vine_hv_per_totv_df.vine=='Y')

In [84]:
vine_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|
|R1OB4GCKEYGV9I|          4|           61|         90|   Y|                N|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|
|R1FNVUXPU63WOZ|          4|           43|         48|   Y|                N|
|R25XGG2G12SE1Z|          4|           20|         23|   Y|                N|
| R3JKU4HRDFZDH|          4|           27|         30|   Y|     

In [112]:
# count how many surveys are by VINE members and store as a variable
vinememberreviewscount=vine_y_df.count()

In [113]:
vinememberreviewscount

47

In [87]:
# how many 5 star reviews were given for vine members
vine_y_df.filter(vine_y_df.star_rating==5).count()

15

In [88]:
# create a dataframe for reviews by non-vine members [5 pts]
vine_n_df=vine_hv_per_totv_df.filter(vine_hv_per_totv_df.vine=='N')

In [89]:
#show the dataframe for reviews by non-vine members
vine_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

In [114]:
# how many non-vine member reviews were there
nonvinememberreviews=vine_n_df.count()

In [115]:
nonvinememberreviews

8362

In [109]:
# count how many 5 star reviews were given for non-vine members
vine_n_df.filter(vine_n_df.star_rating==5).count()

4332

In [93]:
# total reviews after filtering for count greater than 20 and helpful_votes more than 50% was 8373 with 4336 being 5 star or 52% overall

# 8362 non-vine reviews  with 4332 being 5 star reviews or about 52%
# 47 vine reviews with 15 being 5 star reviews or about 32%



In [70]:
summary_df=vine_hv_per_totv_df.groupBy("vine","star_rating").agg({"star_rating":"count"})

In [72]:
# count up the 5 star ratings for both member and non-vine member reviews
summary_df.filter(summary_df.star_rating==5).show()

+----+-----------+------------------+
|vine|star_rating|count(star_rating)|
+----+-----------+------------------+
|   N|          5|              4332|
|   Y|          5|                15|
+----+-----------+------------------+



In [116]:
# calculate the percentage of 5 star reviews by vine members [15 pts]
summary_vine_member_df=summary_df.withColumn('Percentage',summary_df['count(star_rating)']/vinememberreviewscount*100).filter((summary_df.star_rating==5) & (summary_df.vine=='Y'))

In [117]:
summary_vine_member_df.show()

+----+-----------+------------------+------------------+
|vine|star_rating|count(star_rating)|        Percentage|
+----+-----------+------------------+------------------+
|   Y|          5|                15|31.914893617021278|
+----+-----------+------------------+------------------+



In [118]:
summary_vine_member_df2=summary_vine_member_df.withColumn('Total Vine Member Reviews',vinememberreviewscount*summary_vine_member_df.star_rating/summary_vine_member_df.star_rating)

In [119]:
summary_vine_member_df2.show()

+----+-----------+------------------+------------------+-------------------------+
|vine|star_rating|count(star_rating)|        Percentage|Total Vine Member Reviews|
+----+-----------+------------------+------------------+-------------------------+
|   Y|          5|                15|31.914893617021278|                     47.0|
+----+-----------+------------------+------------------+-------------------------+



In [124]:
# calculate the percentage of 5 star reviews by non-vine members [15 pts]
summary_nonvine_member_df=summary_df.withColumn('Percentage',summary_df['count(star_rating)']/nonvinememberreviews*100).filter((summary_df.star_rating==5) & (summary_df.vine=='N'))

In [125]:
summary_nonvine_member_df.show()

+----+-----------+------------------+-----------------+
|vine|star_rating|count(star_rating)|       Percentage|
+----+-----------+------------------+-----------------+
|   N|          5|              4332|51.80578808897393|
+----+-----------+------------------+-----------------+



In [126]:
summary_nonvine_member_df2=summary_nonvine_member_df.withColumn('Total Non-Vine Member Reviews',nonvinemembersurveys*summary_vine_member_df.star_rating/summary_vine_member_df.star_rating)

In [127]:
summary_nonvine_member_df2.show()

+----+-----------+------------------+-----------------+-----------------------------+
|vine|star_rating|count(star_rating)|       Percentage|Total Non-Vine Member Reviews|
+----+-----------+------------------+-----------------+-----------------------------+
|   N|          5|              4332|51.80578808897393|                       8362.0|
+----+-----------+------------------+-----------------+-----------------------------+

